In [7]:
import sys
sys.path.append('/usr/local/lib/python3.9/site-packages/')
import os
import yfinance as yf

import bisect
import math
import matplotlib as plt
import pandas as pd
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
import numpy as np
import copy
import json
import lxml
import time 
from lxml import objectify 
allsymbols=(list(pd.read_csv("nasdaq_screener_1620782407101.csv").loc[:]["Symbol"].values))



In [ ]:
symsubset=allsymbols[5000:6000]
print(symsubset)
totalwrite=0
for isym,sym in enumerate(symsubset):
    try:
        curt = yf.Ticker(sym)
        hist = curt.history(period="max")
        print (isym,"/",len(symsubset),hist.shape)
        if (hist.shape[0]<2000):
            print("not enough data")
            continue
        else:
            totalwrite+=1
        hist.to_json("stockdir/"+sym+".json")

    except:
        "Failed"
print("Wrote",totalwrite,"out of",len(symsubset))

In [8]:
from shutil import copyfile

alldown=os.listdir("filtered")
alldays=[]
dayrange=3000
buffer=30
skipped=0
ran=0
filtersave=True
ncorr=50
allopen=pd.DataFrame()
for ialld,alld in enumerate(alldown):
    symname=alld.replace(".json","")
    #print (alld)
    if ialld%50==0:
        print (ialld,"/",len(alldown))
    #if ialld>50:
     #   break
    try:
        curdf=pd.read_json("stockdir/"+alld)
    except:
        print("Err")
        continue
    alldays.append(curdf.shape[0])
    meandist=[]
    if (curdf.shape[0]>dayrange):


        curdf=curdf.iloc[-(dayrange+buffer):-buffer,:]


        mergeframe=(curdf.loc[:,"Open"]+curdf.loc[:,"Close"])*0.5
        highframe=curdf.loc[:,"High"]
        lowframe=curdf.loc[:,"Low"]
        avedist=(mergeframe-lowframe)*(highframe/lowframe)
        print(avedist)
        mergeframe.name=symname
        if ran==0:
                allopen=mergeframe
                runningindex= (curdf.iloc[-dayrange:,:].index)
                curdf=curdf.loc[runningindex,:]
        else:
                if (curdf.index.difference(runningindex).shape[0]>0):
                    print("date mismatch, skipping...")
                    continue
                else:   
                    try:
                        curdf=curdf.loc[runningindex,:]
                    except:
                        print("Unknown Error")
                        continue

                allopen=pd.concat((allopen,mergeframe),axis=1) 



        if (filtersave):
            curdf.to_json("stockdir/filtered/"+alld)
        #print (curdf.shape)
        ran+=1
    else:
        skipped+=1
        continue
covmat = allopen.corr()
covmatabs=covmat.abs()
for col in covmat.columns:
    absort=covmatabs.loc[col].sort_values()[-(ncorr+1):-1].index
    if (filtersave):
        covmat.loc[col][absort].to_json("stockdir/filtered/corr_"+col+".json")
plt.imshow(covmatabs)


#print(skipped/len(alldown))
#plt.hist(alldays)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'stockdir'

In [52]:
nppalld=np.array(alldays)
print(len(nppalld[nppalld>(3000)])/len(nppalld))


0.3719758064516129
